## Initial Setup and Importing Dataset

In [24]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import math
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

df = pd.read_csv("./Assignment3-UnknownData.csv")
# unknown_df = pd.read_csv("./Assignment3-UnknownData.csv")

## Remove Redundant or Derived Attributes
The following code creates a dictionary (object in other languages) of each attribute that is a code for something (see attribute description). It then uses the code as the key in a nested object and nests another object within that for all the descriptions that had that code along with the count.

In [1]:
dfObj = {}
attributes = df.columns
for y in range(18, 32, 2):
    dfObj[attributes[y]] = {}
    
dfObj["HEAT"] = {}

for row in df.values:
    for y in range(18, 32, 2):
        dfObj[attributes[y]][str(row[y])] = {}
        
    dfObj["HEAT"][str(row[3])] = {}
        
for row in df.values:
    for y in range(18, 32, 2):
        if str(row[y+1]) in dfObj[attributes[y]][str(row[y])]:
            dfObj[attributes[y]][str(row[y])][str(row[y+1])] += 1
        else:
            dfObj[attributes[y]][str(row[y])][str(row[y+1])] = 1
            
    if str(row[4]) in dfObj["HEAT"][str(row[3])]:
        dfObj["HEAT"][str(row[3])][str(row[4])] += 1
    else:
        dfObj["HEAT"][str(row[3])][str(row[4])] = 1

for attribute in dfObj:
    print(attribute)
    total = 0
    for code in dfObj[attribute]:
        print(code, len(dfObj[attribute][code]), dfObj[attribute][code])
        for count in dfObj[attribute][code]:
            total += dfObj[attribute][code][count]
    print(total)

In [25]:
dfObj = {}
attributes = df.columns
# Skip contains the attributes that if missing, the row should be not included
skip = ["HEAT", "STYLE", "STRUCT", "GRADE", "CNDTN", "EXTWALL", "ROOF", "INTWALL", "USECODE", "BLDG_NUM", "SALE_NUM", "AC"]
# nonRatio array is an array of all columns that are numerical but not a ratio type attribute
nonRatio = ["AYB", "YR_RMDL", "EYB", "SALEDATE", "GIS_LAST_MOD_DTTM", "PRICE"]
medians = []

In [26]:
def isSkip(i):
    isSkip = False
    for attribute in skip:
        if attribute == attributes[i]:
            isSkip = True
            break
    return isSkip

def isNonRatio(i):
    isNonRatio = False
    for attribute in nonRatio:
        if attribute == attributes[i]:
            isNonRatio = True
            break
    return isNonRatio

Must add fix missing data before removing redundant attributes

In [27]:
for attribute in df.columns:
    dfObj[attribute] = []
    medians.append(df[[attribute]].median().values)

i = 0
    
for row in df.values:
    newRow = []
    ignoreRow = False
    
    for i in range(0, len(row)):
        value = None
        
        if row[i] != row[i]: # nan (empty cell) check
            if (isSkip(i)):
                ignoreRow = True
                i += 1
                break

            if (isNonRatio(i)):
                value = 0

            if (len(medians[i]) != 0):  # Was the median computed for that attribute
                if value != 0:
                    value = medians[i][0]
            else:
                ignoreRow = True
                i += 1
                break

        else:
            value = row[i]

        newRow.append(value)
    
    if (not ignoreRow):
        for y in range(0, len(attributes)):
            dfObj[attributes[y]].append(newRow[y])
            
print("Count", i)
pd.DataFrame(dfObj).to_csv("test3.csv", index=False)

Count 29


In [6]:
from datetime import datetime
dateAttrs = {
    "sale_year": [],
    "sale_month": [],
    "sale_day": [],
    "gis_year": [],
    "gis_month": [],
    "gis_day": [],
}

for x in df["SALEDATE"].values:
    if (x == 0):
        dateAttrs["sale_year"].append(0)
        dateAttrs["sale_month"].append(0)
        dateAttrs["sale_day"].append(0)
    else:
        date = datetime.strptime(x[:-5], "%Y-%m-%dT%H:%M:%S")
        dateAttrs["sale_year"].append(date.year)
        dateAttrs["sale_month"].append(date.month)
        dateAttrs["sale_day"].append(date.day)
        
for x in df["GIS_LAST_MOD_DTTM"].values:
    if (x == 0):
        dateAttrs["gis_year"].append(0)
        dateAttrs["gis_month"].append(0)
        dateAttrs["gis_day"].append(0)
    else:
        date = datetime.strptime(x[:-5], "%Y-%m-%dT%H:%M:%S")
        dateAttrs["gis_year"].append(date.year)
        dateAttrs["gis_month"].append(date.month)
        dateAttrs["gis_day"].append(date.day)

pd.DataFrame(dateAttrs).to_csv("dates.csv", index=False)

In [7]:
dates_df = pd.read_csv("./dates.csv")
gis = {
    "year": [],
    "month": [],
    "day": []
}

for year in dates_df["gis_year"].values:
    try:
        gis["year"].index(year)
    except:
        gis["year"].append(year)
        
for month in dates_df["gis_month"].values:
    try:
        gis["month"].index(month)
    except:
        gis["month"].append(month)
        
for day in dates_df["gis_day"].values:
    try:
        gis["day"].index(day)
    except:
        gis["day"].append(day)
        
print(gis)

{'year': [2018], 'month': [7], 'day': [22]}


In [23]:
formattedDf = {}
for attribute in df.columns:
    if attribute != "QUALIFIED":
        formattedDf[attribute] = unknown_df[attribute]
    
pd.DataFrame(formattedDf).to_csv("test4.csv", index=False)